<a href="https://colab.research.google.com/github/AamnaMir/PRODIGY_ML_03/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

100%|██████████| 1.06G/1.06G [00:24<00:00, 47.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1


In [2]:
import scipy
import csv as csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk
from numpy import array
import cv2
import os
import skimage as ski
import random
from PIL import Image
from PIL import ImageFilter
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
import re
from PIL import Image
# Import the Resampling enum for image resizing
from PIL.Image import Resampling

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

#get full dataset
TRAIN_DIR = '/root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1/train/'
CAT_DIR = os.path.join(TRAIN_DIR, 'cats')
DOG_DIR = os.path.join(TRAIN_DIR, 'dogs')

# Get image file paths
cat_images = [os.path.join(CAT_DIR, file) for file in os.listdir(CAT_DIR) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
dog_images = [os.path.join(DOG_DIR, file) for file in os.listdir(DOG_DIR) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Determine the minimum count to balance dataset
min_count = min(len(cat_images), len(dog_images), 2500)  # Max 2500 per class

# Sample equal number of images from each class
cat_samples = random.sample(cat_images, min_count)
dog_samples = random.sample(dog_images, min_count)

# Combine balanced dataset
train_images = cat_samples + dog_samples
random.shuffle(train_images)  # Shuffle to mix classes

print(f"Number of cat images: {len(cat_samples)}")
print(f"Number of dog images: {len(dog_samples)}")
print(f"Total training images: {len(train_images)}")

TEST_DIR = '/root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1/test/'
TEST_CAT_DIR = os.path.join(TEST_DIR, 'cats')
TEST_DOG_DIR = os.path.join(TEST_DIR, 'dogs')
test_images = [os.path.join(root, file) for root, _, files in os.walk(TEST_DIR) for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Function to extract HOG features
def extract_hog_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features, _ = ski.feature.hog(gray, pixels_per_cell=(16, 16), cells_per_block=(2, 2),
                                  orientations=9, block_norm='L2-Hys', visualize=True)
    return features

# Prepare dataset
data, labels = [], []
for img_path in train_images:
    image = cv2.imread(img_path)
    if image is not None:
        image = cv2.resize(image, (64, 64))
        features = extract_hog_features(image)
        data.append(features)
        labels.append(0 if "cat" in os.path.basename(img_path) else 1)  # Corrected label assignment

# Convert to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Verify class distribution
unique, counts = np.unique(labels, return_counts=True)
print(f"Class Distribution after feature extraction: {dict(zip(unique, counts))}")  # Should show {0: X, 1: X}

# Split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train SVM model
svm_model = LinearSVC(dual=False, max_iter=5000)
svm_model.fit(X_train, y_train)

# Predict on test set
y_pred = svm_model.predict(X_test)

# Evaluate model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Model Accuracy: {accuracy:.4f}")


Number of cat images: 2500
Number of dog images: 2500
Total training images: 5000
Class Distribution after feature extraction: {np.int64(0): np.int64(2500), np.int64(1): np.int64(2500)}
SVM Model Accuracy: 0.7060
